In [47]:
import boto3
import pandas as pd
from io import StringIO, BytesIO #buffer inmemory to readcsv
from datetime import datetime, timedelta

In [48]:
arg_date = '2022-01-28'
src_format = '%Y-%m-%d'
src_bucket = 'deutsche-boerse-xetra-pds'
trg_bucket = 'xetra-29012022'
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume']
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'


In [49]:
arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)

In [50]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(src_bucket)
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date() >= arg_date_dt]

In [51]:
#bucket_obj1 = bucket.objects.filter(Prefix='2021-03-15')
#bucket_obj2 = bucket.objects.filter(Prefix='2021-03-16')
#objects = [obj for obj in bucket_obj1] + [obj for obj in bucket_obj2]

objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date() >= arg_date_dt]

In [52]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=",")
    return df
df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)

In [53]:
df_all.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [54]:
df_all = df_all.loc[:, columns]

In [55]:
df_all.dropna(inplace=True)

## Get opening price per ISIN and per day 

In [56]:
df_all['opening_price']= df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

## Get closing price per ISIN and day

In [57]:
df_all['closing_price']= df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

## Aggregations

In [58]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

## Percent Change Prev Closing 

In [59]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [60]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price'])/ df_all['prev_closing_price'] * 100

In [61]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [62]:
df_all = df_all.round(decimals=2)

In [63]:
df_all = df_all[df_all.Date >= arg_date]

## Write to S3

In [64]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket(trg_bucket)
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='xetra-29012022', key='xetra_daily_report_20220130_121242.parquet')

## Read the uploaded file

In [65]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20220129_115235.parquet
xetra_daily_report_20220130_121242.parquet


In [66]:
parquet_obj = bucket_target.Object(key="xetra_daily_report_20220129_115235.parquet").get().get('Body').read()
data = BytesIO(parquet_obj)
df_report = pd.read_parquet(data)

In [67]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-28,38.05,37.00,37.00,38.05,456,0.00
1,AT00000FACC2,2022-01-28,7.66,7.52,7.52,7.66,610,-1.31
2,AT0000606306,2022-01-28,25.02,24.66,24.66,25.10,213,-1.67
3,AT0000609607,2022-01-28,12.56,12.56,12.56,12.56,0,0.48
4,AT0000644505,2022-01-28,108.20,108.60,108.20,108.80,125,-0.18
...,...,...,...,...,...,...,...,...
3163,XS2265370234,2022-01-28,20.78,21.03,20.41,21.03,3697,-0.65
3164,XS2284324667,2022-01-28,36.27,36.50,36.09,36.50,5903,0.91
3165,XS2314659447,2022-01-28,8.77,8.58,8.57,8.77,1037,-2.72
3166,XS2314660700,2022-01-28,20.33,20.25,20.05,20.33,58,-0.34
